In [1]:
import torch

from pulser_diff.pulser import Sequence, Pulse, Register
from pulser_diff.pulser.devices import MockDevice
from pulser_diff.pulser.waveforms import BlackmanWaveform, RampWaveform

from pulser_diff.model import QuantumModel

### Toy optimization problem

In [2]:
# create register
reg = Register.rectangle(1, 2, spacing=8, prefix="q")

In [3]:
# create sequence and declare channels
seq = Sequence(reg, MockDevice)
seq.declare_channel("rydberg_global", "rydberg_global")

In [4]:
# define pulse parameters
omega = torch.tensor([5.0], requires_grad=True)
area = torch.tensor([torch.pi], requires_grad=True)

In [5]:
# declare sequence variables
omega_param = seq.declare_variable("omega")
area_param = seq.declare_variable("area")

In [6]:
# create pulses
pulse_const = Pulse.ConstantPulse(1000, omega_param, 0.0, 0.0)
amp_wf = BlackmanWaveform(800, area_param)
det_wf = RampWaveform(800, 5.0, 0.0)
pulse_td = Pulse(amp_wf, det_wf, 0)

# add pulses
seq.add(pulse_const, "rydberg_global")
seq.add(pulse_td, "rydberg_global")

In [7]:
# create quantum model from sequence
trainable_params = {"omega": omega, "area": area}
model = QuantumModel(seq, trainable_params, sampling_rate=0.5, solver="krylov")

# list trainable parameters of the model
print()
for name, param in model.named_parameters():
    print(name)
    print(param)
    print('-------')


trainable_params.area
Parameter containing:
tensor([3.1416], requires_grad=True)
-------
trainable_params.omega
Parameter containing:
tensor([5.], requires_grad=True)
-------


In [8]:
# define loss function and optimizer
loss_fn = torch.nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-1)

In [9]:
# print initial expectation value as a result of simulating initial sequence
_, init_exp_val = model.expectation()
print("Initial expectation value:", init_exp_val[-1])
print()

# optimize model parameters so that the final output expectation value matches the predefined value
epochs = 20
target_value = torch.tensor(-0.5)
for t in range(epochs):
    # calculate prediction and loss
    evaluation_times, exp_val = model.expectation()
    loss = loss_fn(exp_val.real[-1], target_value)

    # backpropagation
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

    # update sequence with changed pulse parameter values
    model.update_sequence()

    print(f"loss: {loss:>7f}")

# print expectation value with optimized model
_, init_exp_val = model.expectation()
print()
print("Optimized expectation value:", init_exp_val[-1])
print()

Initial expectation value: tensor(-1.2782+0.j, grad_fn=<SelectBackward0>)

loss: 0.605560
loss: 0.433920
loss: 0.252128
loss: 0.119507
loss: 0.050335
loss: 0.020542
loss: 0.008450
loss: 0.003535
loss: 0.001501
loss: 0.000645
loss: 0.000280
loss: 0.000122
loss: 0.000053
loss: 0.000023
loss: 0.000010
loss: 0.000005
loss: 0.000002
loss: 0.000001
loss: 0.000000
loss: 0.000000

Optimized expectation value: tensor(-0.5003+0.j, grad_fn=<SelectBackward0>)



We can print the optimized values of amplitude of the constant pulse `omega` and the area of the Blackman pulse `area`:

In [10]:
print()
for name, param in model.named_parameters():
    print(name)
    print(param)
    print('-------')


trainable_params.area
Parameter containing:
tensor([2.7814], requires_grad=True)
-------
trainable_params.omega
Parameter containing:
tensor([4.4623], requires_grad=True)
-------


The values of both parameters were optimized in order to minimize the provided loss function